In [11]:
# %pip install --upgrade pip
# %pip install boto3>=1.34.74 --force-reinstall --quiet
# %pip install botocore --force-reinstall --quiet
# %pip install sqlalchemy==2.0.0 --quiet
# %pip install langchain --force-reinstall --quiet

In [1]:
kb_id = "<redacted>"

In [2]:
import boto3
import pprint
from botocore.client import Config
import json

pp = pprint.PrettyPrinter(indent=2)
session = boto3.session.Session()
region = session.region_name
bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 0})
bedrock_client = boto3.client('bedrock-runtime', region_name = region)
bedrock_agent_client = boto3.client("bedrock-agent-runtime",
                              config=bedrock_config, region_name = region)
print(region)

us-east-1


In [5]:
def retrieve(query, kbId, numberOfResults=5):
    return bedrock_agent_client.retrieve(
        retrievalQuery= {
            'text': query
        },
        knowledgeBaseId=kbId,
        retrievalConfiguration= {
            'vectorSearchConfiguration': {
                'numberOfResults': numberOfResults,
                 "filter": {
                            'andAll':[
                                {
                                "greaterThan": {
                                "key": "n_ingredients",
                                "value": 2
                                }
                            },
                                {
                            "lessThan": {
                                "key": "minutes",
                                "value": 30
                            }
                                }
                            ]
                        }
                # 'overrideSearchType': "HYBRID", # optional
            }
        }
    )

In [7]:
query = "Can you give me quick breakfast menu?"
response = retrieve(query, kb_id, 3)
retrievalResults = response['retrievalResults']
pp.pprint(retrievalResults)

[ { 'content': {'text': 'quick and easy.'},
    'location': {'s3Location': {'uri': 's3://test-kb1/1085.txt'}, 'type': 'S3'},
    'metadata': { 'minutes': '25',
                  'n_ingredients': '3',
                  'name': '2  step pasta sensation',
                  'steps': "['toss hot pasta with cheese and dressing', "
                           '"that\'s it that\'s all !"]',
                  'x-amz-bedrock-kb-source-uri': 's3://test-kb1/1085.txt'},
    'score': 1.3354759},
  { 'content': { 'text': 'this is a fastest low-carb mocha you can make '
                         'without buying a lot of expensive diet-sugars.  the '
                         'nesquick tastes just like sugary nesquick.'},
    'location': {'s3Location': {'uri': 's3://test-kb1/809.txt'}, 'type': 'S3'},
    'metadata': { 'minutes': '1',
                  'n_ingredients': '3',
                  'name': '10 carb mocha',
                  'steps': '[\'combine the above\', "this can be easily made '
            

In [8]:
# fetch context from the response
def get_contexts(retrievalResults):
    contexts = []
    for retrievedResult in retrievalResults: 
        contexts.append(retrievedResult['content']['text'])
    return contexts

In [9]:
contexts = get_contexts(retrievalResults)
# pp.pprint(contexts)

In [10]:
# from langchain.llms.bedrock import Bedrock
import langchain
from langchain_community.chat_models.bedrock import BedrockChat
from langchain.retrievers.bedrock import AmazonKnowledgeBasesRetriever
modelId = 'anthropic.claude-3-sonnet-20240229-v1:0'
llm = BedrockChat(model_id=modelId, 
                  client=bedrock_client)

In [13]:
query = "Could you provide me a quick breakfast menu?"
retriever = AmazonKnowledgeBasesRetriever(
        knowledge_base_id=kb_id,
        retrieval_config={            'vectorSearchConfiguration': {
                'numberOfResults': 3,
                 "filter": {
                            'andAll':[
                                {
                                "greaterThan": {
                                "key": "n_ingredients",
                                "value": 2
                                }
                            },
                                {
                            "lessThan": {
                                "key": "minutes",
                                "value": 30
                            }
                                }
                            ]
                        }
                # 'overrideSearchType': "HYBRID", # optional
            }                          },

    )
docs = retriever.get_relevant_documents(
        query=query
    )
# pp.pprint(docs)

In [14]:
from langchain.prompts import PromptTemplate

PROMPT_TEMPLATE = """
Human: You have great knowledge about food,so provide answers to questions by using fact based and statistical information when possible. 
Use the following pieces of information to provide a concise answer to the question enclosed in <question> tags. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
<context>
{context}
</context>

<question>
{question}
</question>

The response should be specific .

Assistant:"""
claude_prompt = PromptTemplate(template=PROMPT_TEMPLATE, 
                               input_variables=["context","question"])

In [15]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": claude_prompt}
)

In [16]:
answer = qa.invoke(query)
pp.pprint(answer)

{ 'query': 'Could you provide me a quick breakfast menu?',
  'result': 'Based on the provided context, here is a concise suggested quick '
            'breakfast menu:\n'
            '\n'
            '<answer>\n'
            'A healthy breakfast smoothie made with a banana, dates for '
            'natural sweetness, and other ingredients you have on hand like '
            'yogurt, milk, nut butter, etc.\n'
            '\n'
            'Or, if you prefer something savory, boiled or scrambled eggs with '
            'toast made from sliced bread or pasta like the context mentions '
            '"if you\'ve got a kitchen you\'ve got pasta..you\'ve got a '
            'meal!"\n'
            '</answer>\n'
            '\n'
            "I've suggested two quick breakfast options - a fruit smoothie or "
            'eggs with toast/pasta, utilizing the ingredients and ideas '
            'mentioned in the given context. Let me know if you need any '
            'clarification or have an addi

In [23]:
prompt = f"""
Human: You are a human resource AI system, and provides answers to questions by using fact based and statistical information when possible. 
Use the following pieces of information to provide a concise answer to the question enclosed in <question> tags. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
<context>
{contexts}
</context>

<question>
{query}
</question>

The response should be specific and use statistics or numbers when possible.

Assistant:"""

In [25]:
# payload with model paramters
messages=[{ "role":'user', "content":[{'type':'text','text': prompt.format(contexts, query)}]}]
sonnet_payload = json.dumps({
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 512,
    "messages": messages,
    "temperature": 0.5,
    "top_p": 1
        }  )

modelId = 'anthropic.claude-3-sonnet-20240229-v1:0' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'
response = bedrock_client.invoke_model(body=sonnet_payload, modelId=modelId, accept=accept, contentType=contentType)
response_body = json.loads(response.get('body').read())
response_text = response_body.get('content')[0]['text']

pp.pprint(response_text)

('Based on the information provided in the context, to order a computer for '
 'your new hire in the US as a manager, you should:\n'
 '\n'
 '1. Partner with the onboarding team to determine if your new hire requires a '
 'laptop or desktop computer. This seems to be the first step before ordering '
 'any IT equipment.\n'
 '\n'
 '2. If a computer is required, you can order it through the Access Services '
 'Catalogue at '
 'https://servicecatalog.azure.bakerhughes.com/usm/wpf?Node=icguinode.catalogitemdetails&Args=10361&ObjectID=10361&NspPath=&searchIn=&searchInstr=Entire%20Catalog. '
 'The catalogue provides options like standard desktop, standard laptop, '
 'executive laptop, engineering laptop, and engineering grade desktop.\n'
 '\n'
 '3. The ordered computer will come pre-loaded with standard software like '
 'Advantage, Envision, etc. A docking station, 1 monitor, mouse and keyboard '
 'will also be shipped to the new hire.\n'
 '\n'
 '4. If any additional hardware like extra monito

In [17]:
prompt2 = f"""
Human: You have great knowledge about food,so provide answers to questions by using fact based and statistical information when possible. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.

The response should be specific and use statistics or numbers when possible.

Assistant:"""
modelId = 'anthropic.claude-3-sonnet-20240229-v1:0'

In [18]:
def retrieve_and_generate(query, kb_id,modelId, numberOfResults=5):
    return bedrock_agent_client.retrieve_and_generate(
        input= {
            'text': query,
        },
        retrieveAndGenerateConfiguration={
        'knowledgeBaseConfiguration': {
            'generationConfiguration': {
                'promptTemplate': {
                    'textPromptTemplate': f"{prompt2} $search_results$"
                }
            },
            'knowledgeBaseId': kb_id,
            'modelArn': modelId,
            'retrievalConfiguration': {
                'vectorSearchConfiguration': {
                    'numberOfResults': numberOfResults,
                    'overrideSearchType': 'HYBRID',
                     "filter": {
                            'andAll':[
                                {
                                "greaterThan": {
                                "key": "n_ingredients",
                                "value": 2
                                }
                            },
                                {
                            "lessThan": {
                                "key": "minutes",
                                "value": 30
                            }
                                }
                            ]
                        },
                }
        }
                    },
        'type': 'KNOWLEDGE_BASE'
    }
    )

In [21]:
query = "Could you suggest me a quick breakfast menu?"
response = retrieve_and_generate(query, kb_id,modelId, numberOfResults=5)
pp.pprint(response['output']['text'])

("Here's a suggested quick and healthy breakfast menu with some facts and "
 'statistics:\n'
 '\n'
 '1. Overnight Oats: Mix 1/2 cup rolled oats with 1/2 cup milk or yogurt, add '
 'fruits like berries or bananas, and let it soak overnight. Oats are a great '
 'source of fiber - one cup of oats contains about 4 grams of fiber which is '
 '16% of the recommended daily intake.\n'
 '\n'
 '2. Greek Yogurt Parfait: Layer Greek yogurt with fresh or frozen fruits and '
 'a sprinkle of granola or nuts. Greek yogurt is an excellent source of '
 'protein - one 6-ounce serving contains around 15-20 grams of protein.\n'
 '\n'
 '3. Whole Grain Toast with Avocado: Toast a slice of whole-grain bread and '
 'top it with mashed avocado, a sprinkle of salt, and pepper. Avocados are '
 'rich in healthy monounsaturated fats and provide nearly 20 vitamins and '
 'minerals.\n'
 '\n'
 '4. Smoothie: Blend together fruits like bananas, berries, spinach or kale, '
 'milk or yogurt, and a spoonful of nut butter o